# Parte III - Word Vectors

E aí, querido leitor. Bem vinde a parte três do meu projeto de NLP. Nessa parte, vamos ver como usar o Word2Vec, que funcionará como os outros modelos aplicados nas outras etapas do projeto mas com uma diferença de que esse captura contextos.
Ele faz isso transformando as palavras em vetores e mapeando o contexto em que elas ocorrem com relação a outras palavras. Vamos começar então! 


## Pré-processamento

Para esta parte do projeto precisaríamos realizar um pré-processamento semelhante ao da primeira parte, porém, por ser uma função que demora muito tempo para rodar acabei optando por guardar o texto já processado em um novo dataframe, por isso vamos apenas ler esse arquivo e partir direto para o Word2Vec.

Caso queira relembrar a função usada [clique aqui](https://github.com/vitoriars/Projeto-Final-NLP/blob/master/Projeto_final_NLP_Parte_I.ipynb) para ver a primeira parte do projeto.

In [ ]:
#importando a biblioteca
import pandas as pd

#lendo o dataframe
df = pd.read_csv('df_clean.csv', index_col=0)
df.head()

,review,sentiment
0,one reviewers mention watch oz episode hook ri...,positive
1,wonderful little production filming technique ...,positive
2,think wonderful way spend time hot summer week...,positive
3,basically family little boy jake think zombie ...,negative
4,petter mattei love time money visually stunnin...,positive


Tudo certo. Aqui o texto já está processado, sem tags html e stopwords, com todas as letras minúsculas e lemmatizado.

## Word2Vec


### Preparando o texto

Chegamos na parte do modelo. Mas primeiro precisamos preparar o nosso texto, isso porque o Word2Vec só recebe como input uma lista de listas, então precisamos transformá-lo. Vamos lá:

In [ ]:
#biblioteca
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
texto = [word_tokenize(row) for row in df['review']]

### Detectando bigramas

Mais uma coisa antes de partirmos para o modelo: precisamos detectar bigramas no texto, fazemos isso para que o world2vec aprenda a diferenciar bigramas em seus contextos. Para isso, vamos utilizar a biblioteca `gensim`:

In [ ]:
from gensim.models.phrases import Phrases, Phraser

Agora passamos o texto onde queremos procurar bigramas e definimos que a palavra precisa aparecer pelo menos 30 vezes para ser considerada um bigrama:

In [ ]:
phrases = Phrases(texto, min_count=2, threshold=30)

In [ ]:
bigram = Phraser(phrases)

Vamos ver se achamos algum bigrama no nosso texto! Vou colocar para procurar o bigrama Tom Cruise, que por ser um ator famoso deve aparecer várias vezes:

In [ ]:
print(bigram['tom cruise is awesome'.split()]) 

['tom_cruise', 'is', 'awesome']


O texto reconheceu Tom Cruise como um bigrama! Sabemos disso porque ele juntou as palavras com um underline (que é como o phrases 'marca' os bigramas em um texto). Vamos então aplicar para que esses bigramas sejam permanentes no nosso texto:

In [ ]:
 sentences = bigram[texto] #tudo certinho

### Ajustando o modelo

Com nosso texto pronto para o input podemos começar a parte do modelo. Bora lá! Vamos importar as bibliotecas:

In [ ]:
import multiprocessing
from gensim.models import Word2Vec

Vamos ajustar alguns parâmetros ao modelo: 
* o primeiro é o `size`, que representa a dimensão do espaço vetorial, normalmente um valor de 50 a 300. Aqui valores maiores exigem mais dados de treinamento, mas podem levar a modelos melhores. Como nosso df não é tão grande (50 mil linhas) vamos ajustá-lo para `100`; 
* Depois vamos ajustar o `min_count` que é a quatidade de vezes que uma palavra precisa aparecer para ser treinada. Palvras que aparecem apenas uma ou duas vezes em um texto de bilhões de palavras provavelmente são insignificantes, então é melhor ignorá-las: aqui vamos colocar `5`, que é o valor padrão para esse parâmetro segundo a documentação;
* No parâmetro `window`, que se trata da distância da palavra atual e a que queremos prever vamos colocar 2.

In [ ]:
w2v = Word2Vec(min_count=4, size=80, window=2) #modelo preparado!

### Tabela de vocabulário

O word2vec pede pela construção de uma tabela de vocabulário, onde ele filtra as palavras importantes e faz algumas contagens. Então vamos construí-la:

In [ ]:
w2v.build_vocab(sentences) #tabela feita

### Treinando o modelo

Vamos agora para o treinamento do modelo:

In [ ]:
w2v.train(sentences, total_examples=w2v.corpus_count, epochs=20, report_delay=1) 

(100018577, 109487960)

## Analisando o resultado

Nosso modelo está pronto! Agora vamos analisar alguns resultados com funções que o word2vec nos proporciona.

### most_similar

Vamos usar essa função para ver quais são algumas palavras similares de acordo com o contexto das reviews:

In [ ]:
w2v.wv.most_similar(positive=["movie"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('film', 0.8933265209197998),
 ('flick', 0.6963775753974915),
 ('movies', 0.692674994468689),
 ('films', 0.5871027112007141),
 ('picture', 0.5818313360214233),
 ('thriller', 0.5563170909881592),
 ('campfire_tales', 0.5511619448661804),
 ('flatliners', 0.5501915216445923),
 ('horror_flick', 0.549239456653595),
 ('slashers', 0.514405369758606)]

Aqui podemos ver que a palavra que mais se relaciona com "movie" é "film", o que faz sentido já que são sinônimos. Em seguida temos várias outras palavras que realmente se relacionam. A palavra mais curiosa é "really" que não parece ter *tanta* relação assim com "movie" (de um ponto de vista humano, não do modelo). Provavelmente essa palavra pode ter sido usada muitos vezes para aumentar o valor do adjetivo que caracteriza o filme, por exemplo, é muito comum dizermos frases como "this movie is **really** good" ou "**really** bad", etc.

In [ ]:
w2v.wv.most_similar(positive=["good"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('decent', 0.7700533866882324),
 ('great', 0.7157161235809326),
 ('bad', 0.6862451434135437),
 ('nice', 0.634381890296936),
 ('cool', 0.613572359085083),
 ('fairly_decent', 0.6080654859542847),
 ('excellent', 0.5941331386566162),
 ('alright', 0.5888383388519287),
 ('okay', 0.5853169560432434),
 ('halfway_decent', 0.58482825756073)]

Aqui exploramos quais palavras mais se relacionam com o adjetivo "good", as mais próximas são "decent" e "great", o que parece estar certo. Também as outras palavras são todas adjetivos, o que faz muito sentido!

In [ ]:
w2v.wv.most_similar(positive=["tom_cruise"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('kevin_costner', 0.635523796081543),
 ('roger_moore', 0.6126630902290344),
 ('gosford_park', 0.6042460203170776),
 ('mr_magoo', 0.5911063551902771),
 ('enemy_state', 0.5860060453414917),
 ('total_recall', 0.5779522657394409),
 ('batman_robin', 0.5756421089172363),
 ('clark_gable', 0.575323224067688),
 ('anthony_hopkins', 0.5689816474914551),
 ('marlon_brando', 0.5679072737693787)]

Podemos ver também que a palavra mais relacionada a Tom Cruise é Kevin Costner.

In [ ]:
w2v.wv.most_similar(positive=["actor"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('actress', 0.8151911497116089),
 ('actors', 0.7201424837112427),
 ('performer', 0.6442574262619019),
 ('comedian', 0.6301604509353638),
 ('dennis_hopper', 0.6170964241027832),
 ('actors_actresse', 0.5975667238235474),
 ('kevin_spacey', 0.5833142399787903),
 ('cast_member', 0.5771658420562744),
 ('actresses', 0.577154278755188),
 ('actresse', 0.5747847557067871)]

Semelhanças com a palavra "actor".

### similarity

Com essa função podemos ver o quão similar são duas palavras, aleatórias ou não:

In [ ]:
w2v.wv.similarity('movie', 'actor')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.31180236

Aqui vemos que as palavras "movie" e "actor" possuem só 28% de semelhança.

In [ ]:
w2v.wv.similarity('good', 'bad')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.68624514

A semelhança entre "good" e "bad" é de apenas 68%.

In [ ]:
w2v.wv.similarity('actress', 'actor')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.81519115

Entre "actor" e "actress" é 81%

### doesnt_match

Com essa função podemos escolher várias palavras e fazer com que o modelo acerte a palavra que não tem relação com a lista. Por exemplo, a seguir montei uma lista com a palavra filme e seus genêros e adicionei o nome Tom Cruise, que é um ator. Vamos ver qual palavra o modelo nos retorna como não pertencente à lista?

In [ ]:
w2v.wv.doesnt_match(['movie', 'drama', 'action','comedy','tom_cruise'])

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'tom_cruise'

Aqui o modelo retorna que Tom Cruise não pertece à lista. Isso faz muito sentido já que é a palavra que menos se relacionam com as outras!

In [ ]:
w2v.wv.doesnt_match(['good', 'bad', 'decent', 'movie', 'great'])

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'movie'

Aqui vemos que em uma lista de adjetivos a palavra 'movie' é a que mais se diferencia, portanto, não pertence a lista.

## Conclusão

Essa foi a última parte do Projeto de NLP. Durante todo o projeto aprendi vários assuntos, tanto básicos, como fazer um pré-processamento simples, quanto mais avançados como implementar diferentes modelos ao meu texto. Vi que temos diferentes maneiras de realizarmos tarefas semelhantes e que precisamos ter essa intuição de que tipo de modelo funcionaria para determinada tarefa. Também aprendi que cada modelo e cada tarefa pode exigir um pré-processamento diferente, por isso alguns conceitos básicos com stopwords, .lower, regex, etc, são tão importantes.

Obrigada novamente a Julia por ter me auxiliado, principalmente durante essa última parte. Amei ter aprendido tanta coisa desde que entrei no grupo e estou animada para aprender cada vez mais!

